In [1]:
import pyarrow.parquet as pq
import numpy as np
import pandas as pd
import pyarrow as pa
import time

In [2]:
from google.colab import files
uploaded = files.upload()

In [3]:
train_url = "/content/drive/MyDrive/IE 7275 Project_YT/Data/train.parquet"
test_url = "/content/drive/MyDrive/IE 7275 Project_YT/Data/test.parquet"

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
train = pq.read_pandas(train_url).to_pandas()
train.dtypes

video_id                          object
title                             object
publishedAt          datetime64[ns, UTC]
channelId                         object
channelTitle                      object
categoryId                         int64
trending_date                     object
tags                              object
view_count                         int64
likes                              int64
dislikes                           int64
comment_count                      int64
thumbnail_link                    object
comments_disabled                   bool
ratings_disabled                    bool
description                       object
id                                object
duration_seconds                 float64
has_thumbnail                       bool
target                           float64
dtype: object

# Preprocessing data
- Drop features from train data set that are not in the test data set “dislikes”, “comment count” 

- Remove rows of ‘Ratings_disabled’ = False

- Do something with rows with same video.id, group_by(trending_time), age of the video, how long the video has been posted ‘trending_date’ - ‘published_at’

- Transform data types:

  ‘trending_time’ to type datetime 

  ‘description’ from (string) to length of description (int)

- Transform ‘tags’ into number of tags and type of tags

- Create new feature to classify whether a video is a ‘Youtube_shorts’

- Process thumbnails (images), run PCA() on images provided


In [6]:
train = train.drop(['likes', 'dislikes', 'comment_count'], axis=1)
train.head()

,video_id,title,publishedAt,channelId,channelTitle,categoryId,trending_date,tags,view_count,thumbnail_link,comments_disabled,ratings_disabled,description,id,duration_seconds,has_thumbnail,target
0,3C66w5Z0ixs,I ASKED HER TO BE MY GIRLFRIEND...,2020-08-11 19:20:14+00:00,UCvtRTOMP2TqYqu51xNrqAzg,Brawadis,22,2020-08-12,brawadis|prank|basketball|skits|ghost|funny vi...,1514614,https://i.ytimg.com/vi/3C66w5Z0ixs/default.jpg,False,False,SUBSCRIBE to BRAWADIS ▶ http://bit.ly/Subscrib...,3C66w5Z0ixs_2020-08-12,NaN,True,0.103596
1,M9Pmf9AB4Mo,Apex Legends | Stories from the Outlands – “Th...,2020-08-11 17:00:10+00:00,UC0ZV6M2THA81QT9hrVWJG3A,Apex Legends,20,2020-08-12,Apex Legends|Apex Legends characters|new Apex ...,2381688,https://i.ytimg.com/vi/M9Pmf9AB4Mo/default.jpg,False,False,"While running her own modding shop, Ramya Pare...",M9Pmf9AB4Mo_2020-08-12,NaN,True,0.061611
2,J78aPJ3VyNs,I left youtube for a month and THIS is what ha...,2020-08-11 16:34:06+00:00,UCYzPXprvl5Y-Sf0g4vX-m6g,jacksepticeye,24,2020-08-12,jacksepticeye|funny|funny meme|memes|jacksepti...,2038853,https://i.ytimg.com/vi/J78aPJ3VyNs/default.jpg,False,False,I left youtube for a month and this is what ha...,J78aPJ3VyNs_2020-08-12,NaN,True,0.173523
3,kXLn3HkpjaA,XXL 2020 Freshman Class Revealed - Official An...,2020-08-11 16:38:55+00:00,UCbg_UMjlHJg_19SZckaKajg,XXL,10,2020-08-12,xxl freshman|xxl freshmen|2020 xxl freshman|20...,496771,https://i.ytimg.com/vi/kXLn3HkpjaA/default.jpg,False,False,Subscribe to XXL → http://bit.ly/subscribe-xxl...,kXLn3HkpjaA_2020-08-12,NaN,True,0.046804
4,VIUo6yapDbc,Ultimate DIY Home Movie Theater for The LaBran...,2020-08-11 15:10:05+00:00,UCDVPcEbVLQgLZX0Rt6jo34A,Mr. Kate,26,2020-08-12,The LaBrant Family|DIY|Interior Design|Makeove...,1123889,https://i.ytimg.com/vi/VIUo6yapDbc/default.jpg,False,False,Transforming The LaBrant Family's empty white ...,VIUo6yapDbc_2020-08-12,NaN,True,0.040753


In [7]:
ratings_disabled = train[train['ratings_disabled']==True]
train = train[train['ratings_disabled']==False]

# create new feature, age of video
train['description_length'] = train['description'].str.len().fillna(0)
train['trending_date'] = pd.to_datetime(train['trending_date']) #transform trending date to datetime object
train['publishedAt'] = train['publishedAt'].dt.tz_localize(None) #get rid of time zone
train['video_age_seconds'] = (train['trending_date'] - train['publishedAt']).dt.total_seconds().astype(int)

# determine whether a video is shorts or not
train['isShorts']=train['title'].str.contains('shorts')

# Transform ‘tags’ into number of tags and type of tags
train.loc[train['tags'] == '[None]', 'tagsCount'] = 0
train.loc[train['tags'] != '[None]', 'tagsCount'] = train['tags'].str.count('\\|')+1

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-doc

In [8]:
train["tags"]

0        brawadis|prank|basketball|skits|ghost|funny vi...
1        Apex Legends|Apex Legends characters|new Apex ...
2        jacksepticeye|funny|funny meme|memes|jacksepti...
3        xxl freshman|xxl freshmen|2020 xxl freshman|20...
4        The LaBrant Family|DIY|Interior Design|Makeove...
                               ...                        
92270    danny gonzalez|funny|commentary|comedy|react|r...
92271                                               [None]
92272                                             10-19-21
92273                                               [None]
92274    Lizzy Capri|lizzy|Team RAR|Carter Sharer|squid...
Name: tags, Length: 91831, dtype: object

In [9]:
for i in range(len(train)):
  x = train['tags'][i]
  if x != '[None]':
    train['tags'][i] = x.split("|")

#x = train['tags'][0].split("|")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


KeyError: ignored

In [10]:
train.columns

x = ['video_id', 'title', 'publishedAt', 'channelId', 'channelTitle',
       'categoryId', 'trending_date', 'tags', 'view_count', 'thumbnail_link',
       'comments_disabled', 'ratings_disabled', 'description', 'id',
       'duration_seconds', 'has_thumbnail', 'description_length',
       'video_age_seconds', 'isShorts', 'tagsCount']
y = ['target']
train[x]

,video_id,title,publishedAt,channelId,channelTitle,categoryId,trending_date,tags,view_count,thumbnail_link,comments_disabled,ratings_disabled,description,id,duration_seconds,has_thumbnail,description_length,video_age_seconds,isShorts,tagsCount
0,3C66w5Z0ixs,I ASKED HER TO BE MY GIRLFRIEND...,2020-08-11 19:20:14,UCvtRTOMP2TqYqu51xNrqAzg,Brawadis,22,2020-08-12,"[brawadis, prank, basketball, skits, ghost, fu...",1514614,https://i.ytimg.com/vi/3C66w5Z0ixs/default.jpg,False,False,SUBSCRIBE to BRAWADIS ▶ http://bit.ly/Subscrib...,3C66w5Z0ixs_2020-08-12,NaN,True,361.0,16786,False,15.0
1,M9Pmf9AB4Mo,Apex Legends | Stories from the Outlands – “Th...,2020-08-11 17:00:10,UC0ZV6M2THA81QT9hrVWJG3A,Apex Legends,20,2020-08-12,"[Apex Legends, Apex Legends characters, new Ap...",2381688,https://i.ytimg.com/vi/M9Pmf9AB4Mo/default.jpg,False,False,"While running her own modding shop, Ramya Pare...",M9Pmf9AB4Mo_2020-08-12,NaN,True,715.0,25190,False,25.0
2,J78aPJ3VyNs,I left youtube for a month and THIS is what ha...,2020-08-11 16:34:06,UCYzPXprvl5Y-Sf0g4vX-m6g,jacksepticeye,24,2020-08-12,"[jacksepticeye, funny, funny meme, memes, jack...",2038853,https://i.ytimg.com/vi/J78aPJ3VyNs/default.jpg,False,False,I left youtube for a month and this is what ha...,J78aPJ3VyNs_2020-08-12,NaN,True,513.0,26754,False,30.0
3,kXLn3HkpjaA,XXL 2020 Freshman Class Revealed - Official An...,2020-08-11 16:38:55,UCbg_UMjlHJg_19SZckaKajg,XXL,10,2020-08-12,"[xxl freshman, xxl freshmen, 2020 xxl freshman...",496771,https://i.ytimg.com/vi/kXLn3HkpjaA/default.jpg,False,False,Subscribe to XXL → http://bit.ly/subscribe-xxl...,kXLn3HkpjaA_2020-08-12,NaN,True,762.0,26465,False,23.0
4,VIUo6yapDbc,Ultimate DIY Home Movie Theater for The LaBran...,2020-08-11 15:10:05,UCDVPcEbVLQgLZX0Rt6jo34A,Mr. Kate,26,2020-08-12,"[The LaBrant Family, DIY, Interior Design, Mak...",1123889,https://i.ytimg.com/vi/VIUo6yapDbc/default.jpg,False,False,Transforming The LaBrant Family's empty white ...,VIUo6yapDbc_2020-08-12,NaN,True,2493.0,31795,False,33.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92270,H1tQhK0n5Qk,thanks for 5 million,2021-11-23 19:13:46,UCSUf5_EPEfl4zlBKZHkZdmw,Danny Gonzalez,23,2021-11-30,danny gonzalez|funny|commentary|comedy|react|r...,536056,https://i.ytimg.com/vi/H1tQhK0n5Qk/default.jpg,False,False,#shorts,H1tQhK0n5Qk_2021-11-30,59.0,True,7.0,535574,False,9.0
92271,3CuO39VL3rk,Mrs. Rogers done F’ed up😅 #shorts #story #come...,2021-11-24 23:48:37,UCqseWqrl7r-hNoWme7hIfTw,Rebecca Rogers (Mrs. Rogers),23,2021-11-30,[None],863236,https://i.ytimg.com/vi/3CuO39VL3rk/default.jpg,False,False,None,3CuO39VL3rk_2021-11-30,58.0,True,0.0,432683,True,0.0
92272,qfDVsuwg0TU,MY NEW KITCHEN REVEAL!!!!! EXCITING,2021-11-23 22:50:19,UCnB4LOZe6Dc5JUH9EoaHhKQ,Bri & Fern,1,2021-11-30,10-19-21,194119,https://i.ytimg.com/vi/qfDVsuwg0TU/default.jpg,False,False,Contracter email: Los3topetes@gmail.comFOLLOW ...,qfDVsuwg0TU_2021-11-30,824.0,True,192.0,522581,False,1.0
92273,HHbF8HaC4w8,Using Reverse Psychology To Get ANYTHING You Want,2021-11-21 21:15:16,UCQNDoTI2g7_daH6zL_9rvWg,Amyy Woahh,23,2021-11-30,[None],24584472,https://i.ytimg.com/vi/HHbF8HaC4w8/default.jpg,False,False,None,HHbF8HaC4w8_2021-11-30,56.0,True,0.0,701084,False,0.0


In [11]:
for i in range(len(train)):
  x = train['tags'][i]
  if isinstance(x, list):
    print(i)

0
1
2
3
4
5
7
8
9
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
32
33
34
35
36
38
40
41
42
43
44
45
46
47
49
50
51
52
53
54
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
74
75
78
79
80
81
82
83
85
86
87
89
90
91
93
94
95
96
97
98
99
101
102
103
104
105
106
107
108
109
110
111
113
114
115
116
118
119
120
121
123
124
125
126
127
129
130
132
133
134
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
175
176
177
178
179
180
181
182
183
185
186
187
189
190
191
192
194
195
196
198
199
201
202
204
205
206
207
208
209
210
213
214
215
216
217
218
219
221
223
224
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
242
243
244
245
246
247
248
250
251
252
254
255
256
257
258
259
260
261
262
263
266
267
269
270
271
273
274
275
276
277
278
280
281
282
283
284
285
286
287
288
289
290
291
292
293
296
297
298
299
300
302
303
304
305
306
307
309
310
311
312
313
314
315
317
318


KeyError: ignored

In [13]:
test = pq.read_pandas(test_url).to_pandas()

In [14]:
print(test.dtypes)

video_id                          object
title                             object
publishedAt          datetime64[ns, UTC]
channelId                         object
channelTitle                      object
categoryId                         int64
trending_date                     object
tags                              object
thumbnail_link                    object
comments_disabled                   bool
ratings_disabled                    bool
description                       object
id                                object
duration_seconds                 float64
has_thumbnail                       bool
isTest                              bool
dtype: object


In [ ]:
import os
train.to_csv(r'/content/drive/MyDrive/IE 7275 Project_YT/Data/train.csv',index = False, header=True)